In [1]:
import re
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
import spacy
import nltk
from nltk import word_tokenize
from nltk import pos_tag
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('averaged_perceptron_tagger')
nltk.download('all')
from collections import Counter
from wordcloud import WordCloud
import urllib.request



[nltk_data] Downloading package punkt to /Users/rakeyser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rakeyser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/rakeyser/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# with open('data.json', 'r') as f:
#     data_json = json.load(f)
data_df = pd.read_csv('data.csv')

/var/folders/r9/d6mb1jqj0pj3qc1lyl8bq1vm0000gn/T/ipykernel_22061/2227293677.py:3: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  data_df = pd.read_csv('data.csv')


In [3]:
data_list = list(data_df["patent_abstract"])
stop_words = set(stopwords.words('english'))

# Text string separated by spaces, with stop words
concat_text_str = ' '.join(data_list).lower().split()

# Text list with non alpha filtered out, with stop words
concat_text_list = [word for word in concat_text_str if word.isalpha()]

# Text list with non alpha filtered out, with stop words removed
concat_text = [word for word in concat_text_str if word.isalpha() and word not in stop_words]

In [4]:
# Used for finding parts of speech words (nouns, verbs, adj, etc.) Very resource intensive, not recommended 
def extract_POS(texts, pos_type, top_n=100):
    
    nlp = spacy.load("en_core_web_sm")
    pos_list = []

    for text in texts:
        text = text.lower()
        word_tokens = word_tokenize(text)
        filtered_text = [word for word in word_tokens if word.casefold() not in stop_words]
        text = " ".join(filtered_text)

        doc = nlp(text)
        
        for token in doc:
            if token.pos_ == pos_type:
                pos_list.append(token.text)

    most_common_pos = [word for word, word_count in Counter(pos_list).most_common(top_n)]

    return most_common_pos

# print("running nouns")
#nouns = extract_POS(data_list, "NOUN", 100)
# print(nouns)
# adjectives = extract_POS(data_list, "ADJ", 100)
# print(adjectives)
# verbs = extract_POS(data_list, "VERB", 100)
# print(verbs)

In [5]:
# print(nouns)
# print(adjectives)
# print(verbs)

In [6]:
def Sort(tuple): 
    tuple.sort(key = lambda a: a[1], reverse = True) 
    return tuple 

def count_word_occurrences(text):
    tuples = []
    unique_words = set(text)
    for word in unique_words:
        count = text.count(word)
        tuples.append((word, count))

    tuples = tuples[:100]

    sorted_tuples = Sort(tuples)
    return sorted_tuples

def count_word_occurrences_unsorted(text):
    word_list = text.split(" ")
    tuples = []
    unique_words = set(word_list)

    for word in unique_words:
        count = word_list.count(word)
        tuples.append((word, count))

    tuples = tuples[:100]
    return tuples

def word_counts(word_list):
    word_counter = Counter(word_list)
    word_counts = list(word_counter.items())

    if len(word_counts) > 100:
        word_counts = word_counts[:100]
    
    sorted_tuples = Sort(word_counts)
    return sorted_tuples

def noun_filter(filtered_words):
    pos_words = pos_tag(filtered_words)
    nouns = [word for word, pos in pos_words if pos in ['NN', 'NNS', 'NNP', 'NNPS']]
    most_common_nouns = Counter(nouns).most_common(100)

    return most_common_nouns

# Create a word cloud from a list of words
def create_word_cloud(words):
    word_frequencies = Counter(words)
    wordcloud = WordCloud(background_color='white').generate_from_frequencies(word_frequencies)

    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.show()

# Create a round word cloud from a provided tuple of words and instances
def create_word_cloud_tuple(tuple_list):
    word_freq_dict = dict(tuple_list)

    x, y = np.ogrid[:300, :300]
    mask = (x - 150) ** 2 + (y - 150) ** 2 > 130 ** 2
    mask = 255 * mask.astype(int)

    wordcloud = WordCloud(width=1920, height=1080, max_words=100, background_color='white', mask=mask, colormap="magma", relative_scaling=.75)
    wordcloud.generate_from_frequencies(word_freq_dict)

    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.show()


In [7]:
occurences_list = word_counts(concat_text_list)
occurences = word_counts(concat_text)

In [8]:
# noun_list = noun_filter(concat_text)

In [9]:
# create_word_cloud(concat_text)

In [10]:
# create_word_cloud_tuple(noun_list)

In [11]:
data_df

,Unnamed: 0,patent_id,patent_title,patent_date,patent_abstract,application_id,application_filing_date,applicant_name,applicant_organization,assignee_name,assignee_org_name,cpc_class_id,cpc_subclass_id,cpc_group_id,filing_date,inventor_name,inventor_city,inventor_state,inventor_country
0,0,8341762,Safety vest assembly including a high reliabil...,2013-01-01,There is provided a safety vest assembly inclu...,12/077841,2008-03-21,NaN,NaN,NaN,NaN,F41_A41_A41,F41H_A41D_A41D,F41H1/02_A41D1/005_A41D31/245,NaN,Alfiero Balzano,Garden Grove,CA,US
1,1,8341763,Reinforcing element,2013-01-01,The invention relates to a reinforcing element...,11/657400,2007-01-24,NaN,NaN,NaN,adidas International Marketing B.V.,A41_A63_A63,A41D_A63B_A63B,A41D19/01588_A63B71/148_A63B2243/0025,NaN,Dirk Meythaler_Erwin Friedrich Saur_Detlef Mue...,Nuremberg_Schwabach_Zwingenberg_Lonnerstadt_Nu...,NaN,DE_DE_DE_DE_DE
2,2,8341764,Calf protector for rowers,2013-01-01,"A calf protector for rowers, comprising a sing...",12/781786,2010-05-17,NaN,NaN,NaN,NaN,A63_A41_A63_A63_A63,A63B_A41D_A63B_A63B_A63B,A63B71/1225_A41D13/0543_A63B69/06_A63B2071/126...,NaN,Alfred Wendell Wilson,Manassas,VA,US
3,3,8341765,Garment for training swimmers,2013-01-01,A training garment is provided for use by a sw...,12/609340,2009-10-30,NaN,NaN,NaN,NaN,A41_A41_A63_A63_A63_A63_A63,A41D_A41D_A63B_A63B_A63B_A63B_A63B,A41D7/00_A41D13/0012_A63B21/0602_A63B2208/03_A...,NaN,Ralph R. Brodbeck,Cincinnati,OH,US
4,4,8341766,Jacket,2013-01-01,A jacket has several features useful for parti...,12/099614,2008-04-08,NaN,NaN,NaN,"Taylor Made Golf Company, Inc.",A41_A41_A41_A41,A41D_A41D_A41D_A41D,A41D3/00_A41D13/0512_A41D27/10_A41D27/18,NaN,Patricia Elizabeth Dahan_Amanda Rodriguez,Temecula_Carlsbad,CA_CA,US_US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278513,278513,8621657,Device and method for protecting an electronic...,2013-12-31,The invention relates to an electronic system ...,13/054164,2009-07-15,Pierre Lacroix,NaN,NaN,Compagnie Industrielle et Financiere D'Ingenie...,G07_G06_G06_G06_G06_G07_H05,G07F_G06F_G06F_G06K_G06Q_G07F_H05K,G07F7/082_G06F21/83_G06F21/86_G06K7/0091_G06Q2...,NaN,Pierre Lacroix,Saint-Péray,NaN,FR
278514,278514,8621658,Magnetic field observation device and magnetic...,2013-12-31,A magnetic-field-observation device and method...,13/819486,2011-09-05,Hitoshi Saito_Satoru Yoshimura,NaN,NaN,Akita University,G01_G01_G01_G01_G01,G01Q_G01Q_G01Q_G01R_G01R,G01Q60/38_G01Q60/50_G01Q60/54_G01R33/022_G01R3...,NaN,Hidefumi Saito_Satoru Yoshimura,Akita_Akita,NaN,JP_JP
278515,278515,8621659,Cantilever for magnetic force microscope and m...,2013-12-31,In a method of manufacturing this cantilever f...,13/462845,2012-05-03,Kaifeng Zhang_Takenori Hirose_Masahiro Watanab...,NaN,NaN,Hitachi High-Technologies Corporation,G01_G01_G01_G01_G01_G01_G01,G01Q_G01Q_G01Q_G01Q_G01Q_G01Q_G01Q,G01Q60/36_G01Q60/42_G01Q60/54_G01Q60/56_G01Q70...,NaN,Mashatoshi Watanabe_Teruaki Tokutomi_Tatuya Hi...,Yokohama_Odawara_Tokyo_Hitachi_Nakano_Yokohama,NaN,JP_JP_JP_JP_JP_JP
278516,278516,8621660,Probe shape evaluation method for a scanning p...,2013-12-31,Provided is a method of evaluating a probe tip...,13/737022,2013-01-09,NaN,Hitachi High-Tech Science Corporation,NaN,HITACHI HIGH-TECH SCIENCE CORPORATION,G01_G01,G01Q_G01Q,G01Q70/10_G01Q40/00,NaN,Maya Watanabe_Hiroumi Momota,Tokyo_Tokyo,NaN,JP_JP
